In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import pysa.emd as emddev
import pysa.eemd as eemddev
import pysa.visualization as plotter
import pysa.utils as utils
import pysa.nhht as nhht
import copy
from multiprocessing import Pool
from timeit import default_timer as timer

from scipy import signal
import scipy
import os
from scipy import fft
from sklearn import svm
from sklearn.model_selection import cross_val_score
from sklearn.preprocessing import StandardScaler
from sklearn.pipeline import make_pipeline
from sklearn.metrics import accuracy_score
from sklearn import preprocessing
from sklearn.model_selection import GridSearchCV
from sklearn.metrics import classification_report
from sklearn.cross_validation import train_test_split
%matplotlib inline
test = 1
plt.rcParams['figure.figsize'] = (12,8)

/usr/local/lib/python3.5/dist-packages/sklearn/cross_validation.py:41: DeprecationWarning: This module was deprecated in version 0.18 in favor of the model_selection module into which all the refactored classes and functions are moved. Also note that the interface of the new CV iterators are different from that of this module. This module will be removed in 0.20.
  "This module will be removed in 0.20.", DeprecationWarning)


In [2]:
def prepare_erps(erps, err_idx):
    new_erps = []
    for i, erp in enumerate(erps):
        new_erps += [prepare_erp(erp, i, err_idx)]
    return new_erps

In [3]:
def prepare_erp(erp, i, err_idx):
    new_erp = copy.deepcopy(erp)
    new_erp = np.delete(new_erp, err_idx, 0)
    new_erp = new_erp.reshape((new_erp.shape[0], new_erp.shape[1] * new_erp.shape[2]))
    return new_erp

In [64]:
def create_feature_vector(features, i):
    feat_vec = np.full((features[0].shape[0], 1), i)
    for feature in features:
        feat_vec = np.hstack((feature, feat_vec))
    return feat_vec

In [55]:
def ERP(amplitudes, divider=2, initial=0, fs=250):
    
    a = np.square(amplitudes)
    # Calculate mean before event
    r_i = a[:, (initial * fs):(divider * fs)].mean(axis=1)
    a_j = np.zeros(a[:, (divider * fs):].shape)
    for r, row in enumerate(a[:, (divider * fs):]):
        a_j[r, :] = [100 * ((a_j_t - r_i[r]) / r_i[r]) for a_j_t in row]
    # Calculate mean after event
    #a_j = a[:, (divider * fs):-(cutoff * fs)].mean(axis=1)
    # Calculate event related potential for given amplitudes
    #erp =  np.mean(100 * ((a_j - r_i) / r_i))
    erp = a_j.mean(axis=1)
    
    return erp

In [53]:
def calculate_ERP(amplitudes):
    erps = np.zeros((amplitudes.shape[0], amplitudes.shape[1], amplitudes.shape[2]))
    for m, measurement in enumerate(amplitudes):
        for s, sensor in enumerate(measurement):
            erps[m,s,:] = ERP(sensor)
    return erps

In [70]:
path = "exp_data/joachim/"

In [71]:
fd1 = np.load(path + 'fd1.npy')
fd2 = np.load(path + 'fd2.npy')
fd3 = np.load(path + 'fd3.npy')

imfs1 = np.load(path + 'imfs1.npy')
imfs2 = np.load(path + 'imfs2.npy')
imfs3 = np.load(path + 'imfs3.npy')

freq1 = np.load(path + 'freq1.npy')
freq2 = np.load(path + 'freq2.npy')
freq3 = np.load(path + 'freq3.npy')

ampl1 = np.load(path + 'ampl1.npy')
ampl2 = np.load(path + 'ampl2.npy')
ampl3 = np.load(path + 'ampl3.npy')

In [72]:
p = Pool(3)
erps1, erps2, erps3 = p.map(calculate_ERP, [ampl1, ampl2, ampl3])
p.close()
p.join()

In [ ]:
features1 = [
    erps1.reshape(erps1.shape[0], erps1.shape[1] * erps1.shape[2])
]

features2 = [
    erps2.reshape(erps2.shape[0], erps2.shape[1] * erps2.shape[2])
]

features3 = [
    erps3.reshape(erps3.shape[0], erps3.shape[1] * erps3.shape[2])
]

In [80]:
feat_vec1 = create_feature_vector(features1, 1)
feat_vec2 = create_feature_vector(features2, 2)
feat_vec3 = create_feature_vector(features3, 3)

In [81]:
np.save(path + 'feat_vec1', feat_vec1)
np.save(path + 'feat_vec2', feat_vec2)
np.save(path + 'feat_vec3', feat_vec3)